This work is by DJOUANI Taqiyeddine and AMIR AIT Habib


<span style="color: yellow; font-style: italic;">CHATGPT Usage is highighted in YELLOW</span>

Double click on me to use this below:

<span style="color: yellow; font-style: italic;"></span>

### <span style="color: #1A7BCA">Excercise 1</span>

**Question 1.1**

Hi, like always we start by importing our libraries.

Note: Not all libraries will be imported in this code section

In [15]:
import math

In [10]:
peers = [1, 8, 14, 21, 32, 38, 42, 48, 51, 56]
Nmax = 64 

Here’s the deal with the successor function: it finds the smallest peer that’s larger than a given key. This is straightforward enough when the key is somewhere in the middle of the range, but the tricky part is handling wrap-around when the key is larger than all the peers. For instance, if my key is 90 and the peers are [10, 30, 70], the successor would wrap around and give me 10

So, how are we going to do it ?

1. **Sort the peers**: sorting the list makes it much easier to find the next larger peer.

2. **Search for the successor**: I loop through the sorted list, and as soon as I find a peer larger than my key, I return it.

3. **Handle the wrap-around**: If no peer is larger than the key, I return the smallest peer, which is the first element in the sorted list.

In [2]:
def successor(key, peers=peers):
    sorted_peers = sorted(peers) # We need to ensure we are checking in order

    for peer in sorted_peers:
        if peer > key:
            return peer
    return sorted_peers[0] # We are returing 1, we wrap around to the smallest peer (first in the sorted list)

The predecessor function flips this logic. Instead of finding the next larger peer

In [5]:
def predecessor(key, peers=peers):
    sorted_peers = sorted(peers)
    
    if key <= sorted_peers[0]:
        return sorted_peers[-1] # We return to the largest peer (It's circle don't forget that)
                                # In case the peer is 1 or anything smaller than that, we should go to the peer before 1 ( whihc is 56)
    
    for i in range(len(sorted_peers)):
        if sorted_peers[i] >= key:
            return sorted_peers[i-1]

In [7]:
print('\n'.join(f"The successor of the key {p} is the peer {successor(p)}." for p in [30, 42, 60]))

The successor of the key 30 is the peer 32.
The successor of the key 42 is the peer 48.
The successor of the key 60 is the peer 1.


In [8]:
print('\n'.join(f"The predecessor of the key {p} is the peer {predecessor(p)}." for p in [37, 1]))

The predecessor of the key 37 is the peer 32.
The predecessor of the key 1 is the peer 56.


**Question 1.2**

**Building the Distributed Hash Table**

Now that the ring structure is set up, the next step is to create the DHT itself.

So each peer gets:
Its successor, its predecessor and its finger table

For each peer, We compute the finger table entries using the formula (peer_id + 2^i) % Nmax, where ``i`` is the position in the finger table. The result is passed to the successor function to map it to the corresponding peer.



In [13]:
def make_dht(peers=peers):
    dht_dict = {}
    m = int(math.log2(Nmax)) # m : is the number of bits to write the keyspace (64)

    for peer in peers:
        finger_table = []

        for i in range(m):
            value = (peer + 2**i) % Nmax
            finger_entry = successor(value)

            if finger_entry not in finger_table:
                finger_table.append(finger_entry)

        dht_dict[peer] = {
            'key': peer,
            'succ': successor(peer),
            'pred': predecessor(peer),
            'finger': finger_table
        }
    return dht_dict

In [16]:
dht = make_dht(peers)
print({peer: {'key': peer, 'succ': info['succ'], 'pred': info['pred'], 'finger': info['finger']} for peer, info in dht.items()})

{1: {'key': 1, 'succ': 8, 'pred': 56, 'finger': [8, 14, 21, 38]}, 8: {'key': 8, 'succ': 14, 'pred': 1, 'finger': [14, 21, 32, 42]}, 14: {'key': 14, 'succ': 21, 'pred': 8, 'finger': [21, 32, 48]}, 21: {'key': 21, 'succ': 32, 'pred': 14, 'finger': [32, 38, 56]}, 32: {'key': 32, 'succ': 38, 'pred': 21, 'finger': [38, 42, 51, 1]}, 38: {'key': 38, 'succ': 42, 'pred': 32, 'finger': [42, 48, 56, 8]}, 42: {'key': 42, 'succ': 48, 'pred': 38, 'finger': [48, 51, 1, 14]}, 48: {'key': 48, 'succ': 51, 'pred': 42, 'finger': [51, 56, 1, 21]}, 51: {'key': 51, 'succ': 56, 'pred': 48, 'finger': [56, 1, 8, 21]}, 56: {'key': 56, 'succ': 1, 'pred': 51, 'finger': [1, 14, 32]}}


### <span style="color: #1A7BCA">Excercise 2</span>

**Question 2.1**

The if-else statement handles whether the interval between pred and key is continuous or wraps around.

1. Case1: If the predecessor's key is smaller than the current peer's key, the range is straightforward. The function checks if the given key falls within this continuous range.

2. Case2: If the predecessor's key is larger than the current peer's key, the range wraps around the end of the keyspace. In this case, the key is valid if it’s either:

    * Greater than or equal to the predecessor’s key (part of the end of the range),
    * Or smaller than the current peer's key (part of the start of the range).

In [17]:
def isincharge(dht_i, key):
    # It is important to understand this:
    # A peer is responsible for a key 'K' if:
    # The key K lies in the interval between the predecessor of p and P itself
    # We can express this as:
    # if predecessor(p) < K <= P, then P is responsible for K.

    current_key = dht_i['key']      #38
    predecessor_key = dht_i['pred'] # from one to 64

    if predecessor_key < current_key:
        return predecessor_key <= key < current_key  # return true, if the valuated condition is correct else return false
    else:
        return key >= predecessor_key or key < current_key

In [18]:
i = 1; k = 56
value = isincharge(dht[i], k)
print(value)

k = 55
value = isincharge(dht[i], k)
print(value)
i = 38
responsible_keys = [k for k in range(Nmax) if isincharge(dht[i], k)]
print(responsible_keys)

True
False
[32, 33, 34, 35, 36, 37]


**Question 2.2**

I’ll admit, I got stuck on this for a while. Initially, my logic broke for edge cases like when the key was equal to the predecessor. I couldn’t figure out why some keys were being skipped until I realized I needed to split the range into two parts for wrap-around cases. 

<span style="color: yellow; font-style: italic;">I asked ChatGPT about handling these kinds of circular ranges, and it suggested breaking the logic into clear subcases instead of trying to cram everything into one if statement. That tip saved me a lot of headaches!</span>

Another Thing: **Misplaced Recursive Call**

Initially, I structured the lookup function to call itself recursively when the key wasn't found on the current peer. Here’s the logic I started with:

1. Responsibility Check:
First, the function checks whether the current peer is responsible for the key using isincharge. If yes, it’s simple—we return the peer.

2. Find the Closest Peer:
If not, the function consults the finger table to locate the closest peer that might have the key. This peer is called closest_peer.

3. Recursive Call:
I thought, “Great! Let’s just hand over the work to closest_peer by calling lookup(closest_peer, key, dht).”

Here’s where the disaster struck. Instead of progressing the search efficiently, I ended up creating a feedback loop. Every recursive call would circle back, eventually returning to the same peer, and the process would repeat indefinitely. The program hung, and I sat there staring at my screen, utterly confused.

**The Fix: Using the Successor**  
<span style="color: yellow; font-style: italic;">CHATGPT gave me an idea about something else but it inspired me to do it this way</span>

The solution was simpler than I expected:

If the closest peer isn’t the key’s successor, I should pass control to the successor directly.
This ensures that every step progresses forward in the ring, breaking the circular dependency caused by incorrect finger table traversal.

In [19]:
def lookup(i, key, dht=dht):
    current_peer = dht[i]
    peer_key = current_peer['key']
    
    # To build a recursion we need a base case
    # this is where the loop should end, otherwise we end up with infinite loop
    #if pred_key < key <= peer_key or (pred_key > peer_key and (key > pred_key or key <= peer_key)):
    if isincharge(dht[i], key):
        return [i] # Why would i add '[]', because otherwise you will sum up numbers (look in the 3rd return)
    
    successor = current_peer['succ']
    predecessor = current_peer['pred']
    # Added after exo 3, to handle the unique situation where dht contains only one peer responsible for everything
    if successor == peer_key and predecessor == peer_key:
        return [i]
    
    finger_table = current_peer['finger']
    # Case 2: Use the finger table to find the closest peer
    for finger in reversed(finger_table): # the reversed function is very important to our search
                                            # if we dont reverse the order = > This could lead to inefficient routing with unnecessary intermediate hops.
                                            # I noticed when i didnt use it that the path was long and it is not prioritizing
                                            # the largest valid finger 
        if finger > peer_key and finger <= key:
            return [i] + lookup(finger, key, dht)
    
    # If no finger is valid, move to the successor
    return [i] + lookup(successor, key, dht)

In [20]:
result = lookup(56, 2, dht)
print(result)

[56, 1, 8]


In [21]:
result = lookup(1, 52, dht)
print(result)

[1, 38, 48, 51, 56]
